**Scenario**: we are working for Atrapalo in Mexico and we want to have a close look at what Skyscanner is doing. One of the hypothesis we want to test is that they are ranking and performing well on some queries where we don't even have a page created. Let's have a look at that! 

**Limitations**: our analysis is 100% based on what is included in sitemap data. We may have situations where data included in our or our competitors' data is not exhaustive. We will ignore this limitations during this tutorial. 

## Setup

In [ ]:
#Import pandas(https://pandas.pydata.org/)
import pandas as pd 
#Import numpy (https://numpy.org/)
import numpy as np 
#Import advertools (https://pypi.org/project/advertools/)
try: 
  import advertools as adv
except:
  !pip install advertools
  import advertools as adv 

## Download sitemaps

In [ ]:
#Load sitemaps
atrapalo = adv.sitemap_to_df('https://www.atrapalo.com.mx/sitemaps/sitemap.xml.gz')
skyscanner = adv.sitemap_to_df('https://www.skyscanner.com.mx/sitemap.xml')

2020-11-19 08:21:51,654 | INFO | sitemaps.py:361 | sitemap_to_df | Getting https://www.atrapalo.com.mx/sitemaps/vue_destinations.xml.gz
2020-11-19 08:21:51,668 | INFO | sitemaps.py:361 | sitemap_to_df | Getting https://www.atrapalo.com.mx/sitemaps/via_categories.xml.gz
2020-11-19 08:21:51,727 | INFO | sitemaps.py:361 | sitemap_to_df | Getting https://www.atrapalo.com.mx/sitemaps/vmh_locations.xml.gz
2020-11-19 08:21:51,800 | WARNING | sitemaps.py:355 | sitemap_to_df | HTTP Error 404: Not Found https://www.atrapalo.com.mx/sitemaps/via_trips.xml.gz
2020-11-19 08:21:52,385 | INFO | sitemaps.py:361 | sitemap_to_df | Getting https://www.atrapalo.com.mx/sitemaps/vue_routes.xml.gz
2020-11-19 08:21:54,231 | INFO | sitemaps.py:361 | sitemap_to_df | Getting https://www.atrapalo.com.mx/sitemaps/hot_hotels_4.xml.gz
2020-11-19 08:21:54,366 | INFO | sitemaps.py:361 | sitemap_to_df | Getting https://www.atrapalo.com.mx/sitemaps/via_locations.xml.gz
2020-11-19 08:21:55,085 | INFO | sitemaps.py:361 | s

In [ ]:
#Quickly check the number of rows & columns returned for both sitemaps 
print('Atrapalo: ',atrapalo.shape)
print('Skyscanner: ',skyscanner.shape)

Atrapalo:  (416097, 7)
Skyscanner:  (578187, 6)


## Filter data

In [ ]:
#In this analysis, we will only analyze the Flight Routes, so let's remove useless rows from our 2 DFs
atrapalo_flight = atrapalo[(atrapalo['sitemap'].str.contains('route'))]
skyscanner_flight = skyscanner[skyscanner['sitemap'].str.contains('route')]

print('Atrapalo: ',atrapalo_flight.shape)
print('Skyscanner: ',skyscanner_flight.shape)

Atrapalo:  (9280, 7)
Skyscanner:  (29168, 6)


## Extract airport codes

In [ ]:
#Let's extract, in both datasets, the city for the origin & the destination

#Atrapalo
atrapalo_flight['From'] = atrapalo_flight['loc'].str.extract('/vuelos/.*_(.*)-.*\.html$')
atrapalo_flight['To'] = atrapalo_flight['loc'].str.extract('/vuelos/.*_.*-(.*)\.html$')

#Skyscanner
skyscanner_flight['From'] = skyscanner_flight['loc'].str.extract('/rutas/(.*)/.*/.*$')
skyscanner_flight['To'] = skyscanner_flight['loc'].str.extract('/rutas/.*/(.*)/.*$')

#Have a preview or what the extracted content look like
skyscanner_flight.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,loc,changefreq,priority,sitemap,download_date,errors,From,To
25665,https://www.skyscanner.com.mx/rutas/aae/bva/an...,weekly,0.3,https://www.skyscanner.com.mx/es_mx_routes_a_1...,2020-11-19 08:22:05.042760+00:00,NaN,aae,bva
25666,https://www.skyscanner.com.mx/rutas/aae/cdg/an...,weekly,0.3,https://www.skyscanner.com.mx/es_mx_routes_a_1...,2020-11-19 08:22:05.042760+00:00,NaN,aae,cdg
25667,https://www.skyscanner.com.mx/rutas/aae/ist/an...,weekly,0.3,https://www.skyscanner.com.mx/es_mx_routes_a_1...,2020-11-19 08:22:05.042760+00:00,NaN,aae,ist
25668,https://www.skyscanner.com.mx/rutas/aae/ista/a...,weekly,0.3,https://www.skyscanner.com.mx/es_mx_routes_a_1...,2020-11-19 08:22:05.042760+00:00,NaN,aae,ista
25669,https://www.skyscanner.com.mx/rutas/aae/mex/an...,weekly,0.3,https://www.skyscanner.com.mx/es_mx_routes_a_1...,2020-11-19 08:22:05.042760+00:00,NaN,aae,mex


## Append both elements to create a matching ID

In [ ]:
#Let's quickly check that our extraction worked correctly for both websites 
#Atrapalo
print('Atrapalo_from: ',atrapalo_flight['From'].isnull().sum())
print('Atrapalo_to: ',atrapalo_flight['To'].isnull().sum())
#Skyscanner
print('Skyscanner_from: ',skyscanner_flight['From'].isnull().sum())
print('Skyscanner_to: ',skyscanner_flight['To'].isnull().sum())


Atrapalo_from:  0
Atrapalo_to:  0
Skyscanner_from:  0
Skyscanner_to:  0


In [ ]:
#Let's build the full route by appending the FROM and the TO codes
atrapalo_flight['Route'] = atrapalo_flight['From']+'-'+atrapalo_flight['To']
skyscanner_flight['Route'] = skyscanner_flight['From']+'-'+skyscanner_flight['To']

2020-11-19 08:22:48,160 | INFO | utils.py:141 | _init_num_threads | NumExpr defaulting to 2 threads.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Let's find the % of routes that Skyscanner has but that Atrapalo doesn't
len(skyscanner_flight[skyscanner_flight['Route'].isin(atrapalo_flight['Route'])==False])/len(skyscanner_flight)

0.9518307734503566

## Load SEMRUSH data

Around 95% of Skyscanner's flight routes don't exist on Atrapalo. It is quite huge but we know that routes like https://www.skyscanner.com.mx/rutas/dad/hnd/da-nang-a-tokio-haneda.html may not have a huge demand in Mexico. 

Our goal is therefore to find valuable routes that we must create if we don't have them yet. Let's use SEMRUSH data to do just that. 

In [ ]:
#Load SEMRUSH data and display the first rows just to be sure that everything went fine 
semrush_skyscanner = pd.read_csv('/content/drive/My Drive/RWST/files/skyscanner_semrush.csv')
#Keep only the columns we need 
semrush_skyscanner = semrush_skyscanner[['URL','Traffic']]
semrush_skyscanner.head()

,URL,Traffic
0,https://www.skyscanner.com.mx/rutas/mx/cn/mexi...,3977
1,https://www.skyscanner.com.mx/rutas/mx/co/mexi...,3143
2,https://www.skyscanner.com.mx/rutas/mx/es/mexi...,3119
3,https://www.skyscanner.com.mx/rutas/mex/pari/c...,2572
4,https://www.skyscanner.com.mx/rutas/mx/ar/mexi...,2479


## Magical part

In [ ]:
#Create a new dataframe with only routes from Skyscanner not included in Atrapalo
missing = skyscanner_flight[skyscanner_flight['Route'].isin(atrapalo_flight['Route'])==False]
#Keep only the information we really need 
missing = missing[['loc','From','To','Route']]
#Change a column name to simplify merge 
missing = missing.rename(columns={'loc': 'URL'})
#Add traffic data 
missing = missing.merge(semrush_skyscanner,on='URL')

missing.shape

(743, 5)

We can see that out of the routes we identified as missing on Atrapalo, only 788 seem to be at least be ranking on a keyword (based on SEMRUSH data). Let's have a look at them and maybe clean a little bit more our file. 

## Final results

In [ ]:
#Some routes are between Mexico (the country) and another one, like https://www.skyscanner.com.mx/rutas/mx/cn/mexico-a-china.html
#These are false positive as Atrapalo is not classifying them as Flight Routes 
#Let's remove these cases 

#Remove routes where the origin is a country 
missing_cleaned = missing.copy()
missing_cleaned = missing_cleaned[missing_cleaned['From'].str.len()==3]
#Remove routes where the destination is a country 
missing_cleaned = missing_cleaned[missing_cleaned['To'].str.len()==3]

missing_cleaned.sort_values(by='Traffic',ascending=False)

,URL,From,To,Route,Traffic
184,https://www.skyscanner.com.mx/rutas/pbc/ntr/pu...,pbc,ntr,pbc-ntr,65
73,https://www.skyscanner.com.mx/rutas/fra/bcn/fr...,fra,bcn,fra-bcn,64
77,https://www.skyscanner.com.mx/rutas/bcn/ibz/ba...,bcn,ibz,bcn-ibz,52
740,https://www.skyscanner.com.mx/rutas/yxx/pvr/ab...,yxx,pvr,yxx-pvr,42
80,https://www.skyscanner.com.mx/rutas/bcn/mah/ba...,bcn,mah,bcn-mah,42
...,...,...,...,...,...
119,https://www.skyscanner.com.mx/rutas/gdl/lhr/gu...,gdl,lhr,gdl-lhr,0
118,https://www.skyscanner.com.mx/rutas/gdl/jfk/gu...,gdl,jfk,gdl-jfk,0
115,https://www.skyscanner.com.mx/rutas/bwi/sfo/ba...,bwi,sfo,bwi-sfo,0
101,https://www.skyscanner.com.mx/rutas/bts/waw/br...,bts,waw,bts-waw,0


We end up with a set of 166 routes that Atrapalo would need to analyze to define whether it is relevant to include them on their Mexican website. The first route in our DataFrame (BUD-MAD) is between two European cities (Budapest & Madrid) and may not be the most important route to add, **nevertheless maybe some Mexican are buying flights from Mexico when they plan an European trip. We'll have to check search volume to confirm this theory and maybe add these routes anyway.**

In [ ]:
#save output
missing_cleaned.to_csv('output.csv')